# Event Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

import csv
import json

import unittest

from common import Printable, projdir

from name import Name
from fuzzy import FuzzyMatch
from entrant import Entrant
from session import Session

from constants import *

## Event Class

Class to manage events

In [2]:
class Event(Printable):
    def __init__(self, path, appConfig, existingNames={}, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        self.year = int(os.path.basename(path))

        self.appConfig = appConfig
        self.existingNames = existingNames
        
        self.fuzzyMatch = FuzzyMatch()

        self.entrants = {}
        self.gt31Ids = {}
        self.sailNos = {}

        self.sessions = {}
        self.runs = {}
        self.numRuns = 0


    def checkName(self, names):
        '''Check if name is a potential typo'''

        for name in [name.strip() for name in names.split('+')]:
            if name in self.existingNames:
                self.existingNames[name].years.append(self.year)
            else:
                nameObj = Name(name)
                for existingName in self.existingNames:
                    existingNameObj = self.existingNames[existingName]
                    if not name.startswith('Sail Number') and self.fuzzyMatch.matchNameObjects(nameObj, existingNameObj) is True:
                        self.logWarning("Similar names - '{}' vs '{}' {}".format(
                            name, existingNameObj.name, existingNameObj.years))

                self.existingNames[name] = nameObj
                nameObj.years = [self.year]


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                    
                    self.checkName(entrant.getValue('Name'))
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))


    def indexSails(self):
        '''Create sail number indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for sailNo in entrant.sailNos:
                if sailNo.upper() not in self.sailNos:
                    self.sailNos[sailNo.upper()] = entrant
                else:
                    raise ValueError('Duplicate sail number \"{}\"'.format(sailNo))
        

    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for gt31Id in entrant.gt31Ids:
                if gt31Id.upper() not in self.gt31Ids:
                    self.gt31Ids[gt31Id.upper()] = entrant
                else:
                    raise ValueError('Duplicate GT-31 ID \"{}\"'.format(gt31Id))
        

    def loadSessions(self):
        '''Load all of the event sessions - absence of config will result in an exception (intentional)'''
        
        dataPaths = sorted(glob.glob(os.path.join(self.path, GPSDATA_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for dataPath in dataPaths:
            configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
            session = Session(self, dataPath, configPath, self.appConfig, verbosity=self.verbosity)

            session.loadConfig()
            session.loadRuns(self.sailNos, self.gt31Ids)

            self.sessions[session.date] = session


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)
        else:
            self.runs[entrantId] = [speedRun]

        self.numRuns += 1


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        for entrantId in self.runs:
            self.runs[entrantId].sort(key=lambda x: x.data[T_SPEED], reverse=True)


    def processEvent(self):
        '''Read entrants from config folder'''

        if self.verbosity >= 1:
            print('Processing {}...'.format(self.year))

        self.loadEntrants()  
        self.indexGt31s()
        self.loadSessions()
        self.sortRuns()

        if self.verbosity >= 1:
            print('All done!\n')

## Unit Tests

A handful of very basic tests

In [3]:
class TestEvent(unittest.TestCase):
    '''Class to test Event class'''
    
    def testLoadEntrants(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.loadEntrants()
        
        # Check the entrants = 68 in entrants.csv
        self.assertEqual(len(event.entrants), 68)


    def testIndexGt31Ids(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.loadEntrants()
        event.indexGt31s()
        
        # Check the entrants = 68 in entrants.csv
        self.assertEqual(len(event.gt31Ids), 68)


    def testProcessEvent(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.processEvent()
        
        # Check the entrants = 68 in entrants.csv and 4 unrecognised GPS
        self.assertEqual(len(event.entrants), 68 + 4)

        # Check the event totals
        self.assertEqual(event.numRuns, 3731)
        self.assertEqual(len(event.runs), 68)
        
        # Check runs are sorted correctly
        for personId in event.runs:
            maxSpeed = 99.999
            for run in event.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [4]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.121s

OK


## All Done!